In [1]:
#site i, #participant j, feature g

from spicy import stats
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

np.random.seed(666)
#Total sample size (sum of all sites) is 5000
N=5000
#the number of features is 1000.
G=10
#10 sites
I=10
#for sample size in each site
alpha=np.random.randint(1, 21, size=I)
p = np.random.dirichlet(alpha)

#sample size of site i
n_i=(N*p).round(0).astype(int)


In [2]:
#This section will be the same for both the homogeneous and heterogeneous versions.

###

alpha_G = np.random.uniform(0, 0.5,G)

Y_I=np.random.uniform(0,0.1,I)
tau_I=stats.invgamma.rvs(a=2, scale=0.5,size=I)

df = pd.DataFrame({'Y_i': Y_I, 'tau_i': tau_I})
gamma_IG=df.apply(lambda x: stats.norm.rvs(loc=x['Y_i'], scale=np.sqrt(x['tau_i']), size=G), axis=1)

lambda_I=stats.gamma.rvs(a=50,scale=50,size=I)
v_I=stats.gamma.rvs(a=50,scale=1,size=I)
df1=pd.DataFrame({'lambda_i':lambda_I,'v_i':v_I})
delta_IG=df1.apply(lambda x:stats.gamma.rvs(a=x['lambda_i']*x['v_i'],scale=x['v_i'],size=G),axis=1)

sigma_G=stats.halfcauchy.rvs(scale=0.2, size=G)

#fixed effects of age and sex, different for different features
theta_age=stats.norm.rvs(loc=0,scale=1,size=G)
theta_sex=stats.norm.rvs(loc=0,scale=1,size=G)
theta_G=pd.DataFrame({"age":theta_age,"sex":theta_sex})
# print(theta)


In [3]:
#sex follows a binomial distribution where p is a random number selected from Uniform(0.2,0.8)
p=[np.random.uniform(0.2,0.8,n) for n in n_i]
sex=[]#it is a list with 10 entries, where each entry contains generated data for one site
age=[]#it is a list with 10 entries, where each entry contains generated data for one site
mu_age=np.random.uniform(10,90,I)
i=0
for n in n_i:
    #covariates sex
    #for site i with n_i samples, the probability of male is equal to female as 50%.
    v1=np.random.binomial(n=1, p=p[i], size=n)
    sex.append(v1)
    #covariate age
    mu=mu_age[i]
    a=mu/3-5
    b=mu/3+5
    sd=np.random.uniform(a,b,1)
    #by reading the median of all boxplots, I guess the mean to be 40 and 10 for sd.
    v2=stats.norm.rvs(loc=mu, scale=sd, size=n)
    age.append(v2)
    i+=1
#standardize
standardized_age = [(a - np.mean(a)) / np.std(a) for a in age]

In [8]:
#fixed effect transform function
def fixed_effect_linear(x, theta):  # x: 2x1 and theta: 2x1
    return np.dot(x.T, theta)  

4999

In [6]:
#simulate features for all sites when assuming linear for fixed effect
data = []
for i in range(I):#for each site
    gamma_i = gamma_IG[i]
    delta_i = delta_IG[i]    
    x = pd.DataFrame({'age': standardized_age[i], 'sex': sex[i]})  # Ensure age[i] and sex[i] are iterable
    J=n_i[i]
    data1 = []
    for j in range(J):#for each participant
        each_participant = []  # generated features for each participant
        x_ij = x.iloc[j, :]

        for g in range(G):#for each feature
            sigma_g = sigma_G[g]
            theta_g = theta_G.iloc[g, :]
            epsilon_ijg = stats.norm.rvs(loc=0, scale=sigma_g, size=1)[0]  
            
            phi = fixed_effect_linear(x_ij, theta_g)
            y_ijg = alpha_G[g] + phi + gamma_i[g] + delta_i[g] * epsilon_ijg

            each_participant.append(y_ijg)

        data1.append(each_participant)

    data1 = pd.DataFrame(data1, columns=[f'feature{g}' for g in range(G)])
    data1['site'] = i
    data.append(data1)

final_data = pd.concat(data, ignore_index=True)
print(final_data)
#this simulated feature values are assumed to be value after standardization


          feature0       feature1       feature2       feature3  \
0    -8.118962e+06  222977.108504   33976.297794 -225038.118863   
1    -1.016681e+07   -1816.123066   -8680.547303 -108501.141285   
2    -1.100164e+07  -51331.049524   35804.422728 -142341.100679   
3     5.909972e+06  -56071.118918   82332.529608  222189.868593   
4     1.571627e+07  176475.528199    6183.053972  -43140.307871   
...            ...            ...            ...            ...   
4994  1.028177e+07 -157759.713126  -71800.232090 -120470.414391   
4995  1.570832e+07  171581.219293  -58832.728700   44192.545595   
4996  2.183070e+07  -67953.902860   17290.709244   48928.987284   
4997 -2.252206e+07 -118628.581579  -88304.999502 -246607.044070   
4998 -1.542151e+06 -107770.379720 -170964.401687   51334.372324   

           feature4      feature5      feature6       feature7      feature8  \
0     -28307.595666  1.662661e+06 -5.788937e+04 -182324.679526  1.072913e+05   
1    -151964.389458  1.445250e+06 -

In [12]:
class MLP(nn.Module):
    #two input features, 1 hidden layer, one output, use sigmoid to transform the weighted sum of the inputs to non-linear
    def __init__(self, input_dim, hidden_dim, output_dim,classification=False):
        super(MLP, self).__init__()
        #self.layers = nn.ModuleList()
        #prev_dim = input_dim
        
        #for hidden_dim in hidden_dims:
        self.hidden_layer = nn.Linear(input_dim, hidden_dim)
        nn.init.normal_(self.hidden_layer.weight, mean=0, std=1)  
        nn.init.normal_(self.hidden_layer.bias, mean=0, std=1)    
       
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        nn.init.normal_(self.output_layer.weight, mean=0, std=1)  
        nn.init.normal_(self.output_layer.bias, mean=0, std=1)    
   
    def forward(self, x):
        #for layer in self.layers:
        x = torch.sigmoid(self.hidden_layer(x))  # sigmoid activation function
        return self.output_layer(x) #output without activation function


def fixed_effect_nonlinear(x):#multi-layer perceptron with logistic activation functions
    X_tensor=torch.tensor(x.values,dtype=torch.float32)#comvert datafrmae to numpy
    input_dim=2
    hidden_dim=1
    output_dim=1
    model=MLP(input_dim,hidden_dim,output_dim)
    model.eval()
    y_simulated=model(X_tensor).detach().numpy()
    return(y_simulated)
# print(len(gamma_ig))
#simulate features for all sites when assuming nonlinear for fixed effect
phi_G = []  
standardized_age1 = np.concatenate(standardized_age)
sex1 = np.concatenate(sex)
# print(sex1)
x = pd.DataFrame({'age': standardized_age1, 'sex': sex1})
# print(x.values)
for g in range(G):  
    d = fixed_effect_nonlinear(x)  
    phi_G.append(d)

#convert phi to be in the format of for each site
phi_I=[]
for i in range(I):
    d=[]
    a=0
    b=0
    for g in range(G):
        phi_g=phi_G[g]
        if a==0:
            a=0
            b=a+n_i[i]
        else:
            a=b
            b=a+n_i[i]
        phi_ig=phi_g[a:b]
        d.append(phi_ig)
    phi_I.append(np.column_stack(d))
# print(phi_I[1].shape)   
# print(n_i[1])
# #print(len(phi[0]))
data = []
for i in range(I):#for each site
    J = n_i[i]  # n_i is same for all sites
    gamma_i = gamma_IG[i]
    delta_i = delta_IG[i]    
    x = pd.DataFrame({'age': standardized_age[i], 'sex': sex[i]})  # Ensure age[i] and sex[i] are iterable
    phi_i=pd.DataFrame(phi_I[i])
    data1 = []
    for j in range(J):#for each participant
        each_participant = []  # generated features for each participant
        x_ij = x.iloc[j, :]

        for g in range(G):#for each feature
            sigma_g = sigma_G[g]
            theta_g = theta_G.iloc[g, :]
            epsilon_ijg = stats.norm.rvs(loc=0, scale=sigma_g, size=1)[0]  
            phi=phi_i.iloc[j,g]            
            y_ijg = alpha_G[g] + phi + gamma_i[g] + delta_i[g] * epsilon_ijg

            each_participant.append(y_ijg)

        data1.append(each_participant)

    data1 = pd.DataFrame(data1, columns=[f'feature{g}' for g in range(G)])
    data1['site'] = i
    data.append(data1)

final_data = pd.concat(data, ignore_index=True)
print(final_data)

#the simulated data varies from negative to positive

          feature0       feature1       feature2       feature3  \
0    -2.289167e+07 -248819.352881  -36321.476385 -158417.460338   
1     2.369526e+06  -15757.708364  -57333.011543   25047.306351   
2    -2.592981e+06  -24526.945976 -100536.566308 -235250.779615   
3    -1.975115e+07  167375.941629  -52780.653793   86174.645079   
4     2.115706e+07   -8851.910204  116300.880804  108753.895947   
...            ...            ...            ...            ...   
4994  3.541844e+05 -159404.845608   22517.904969 -145398.588385   
4995  4.386182e+06  -23280.553037 -105023.833241  389494.062174   
4996  9.349970e+06  109883.953358  104303.351525   57467.370564   
4997 -2.510377e+06   90641.964042   63611.473102  122013.752728   
4998  1.144213e+06  113276.999116    3137.818481  -30354.474391   

           feature4      feature5      feature6       feature7      feature8  \
0    -156485.433852  2.371927e+06 -1.568476e+06  -66190.434259  6.740843e+05   
1     -48925.404830  2.262013e+06 -